### 문제 3-1 : 콘텐츠 분쟁 해결 RAG 시스템

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# PDF 파일 경로 설정
pdf_filepath = '../data/콘텐츠분쟁해결_사례.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

loader = PyPDFLoader(pdf_filepath)
documents = loader.load()

In [4]:
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
pythontext_chunks = pythontext_splitter.split_documents(documents)

In [5]:
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)


In [6]:
vectorstore = FAISS.from_documents(documents, pythonembeddings)

pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)


In [7]:
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)


In [8]:
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""


In [9]:
pythonprompt = PromptTemplate(
    input_variables=["context", "question"],
    template=pythonprompt_template  # 기존 문자열 그대로 사용
)

In [10]:
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": pythonprompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)


In [11]:
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


In [12]:
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


In [13]:
for q in pythontest_questions:
    category = classify_dispute_type(q)
    print(f"\n🟦 질문: {q}")
    print(f"➡️ 분쟁 유형: {category}")

    result = pythonqa_chain.invoke({"query": q})
    
    print("\n🟨 [전문 법률 자문]")
    print(result["result"])
    print("\n📚 [참고된 사례 수]:", len(result["source_documents"]))


🟦 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
➡️ 분쟁 유형: 게임

🟨 [전문 법률 자문]
온라인 게임에서 시스템 오류로 인해 아이템이 사라졌을 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다.

1. **사례 기반 답변**:
   - 제시된 사례 1-가-1(2006)에서는 시스템 오류로 인해 아이템이 소멸된 경우, 계정 명의자가 아닌 신청인의 복구 요청이 거부되었습니다. 게임사의 약관에 따라 계정 명의자가 직접 복구 요청을 해야 한다는 점이 강조되었습니다.
   - 사례 1-가-1(2009)에서는 시스템 오류로 인한 게임머니 소실 주장이 있었으나, 당시 유사한 불만이 접수되지 않았고, 신청인의 사용 내역이 정상적이었기 때문에 시스템 오류로 보기 어렵다는 이유로 복구가 거부되었습니다.

2. **법적 근거 제시**:
   - 일반적으로 온라인 게임 서비스의 약관은 계약의 일부분으로 간주되며, 약관에 명시된 내용이 법적 효력을 가집니다. 따라서 약관에 따라 복구가 불가하다고 명시되어 있다면, 법적으로 이를 따를 수밖에 없습니다.

3. **판례 참조**:
   - 위의 사례들에서 공통적으로 나타나는 것은, 게임사의 약관에 따라 복구 여부가 결정된다는 점입니다. 특히 계정 명의 문제나 시스템 오류의 입증 여부가 중요한 쟁점으로 작용합니다.

4. **실무 가이드**:
   - **1단계**: 게임사의 약관을 확인하여 시스템 오류로 인한 아이템 소실에 대한 복구 정책을 검토합니다.
   - **2단계**: 계정 명의자 여부를 확인하고, 명의자가 직접 복구 요청을 할 수 있도록 조치합니다.
   - **3단계**: 시스템 오류의 발생 여부를 입증할 수 있는 증거(예: 오류 발생 당시의 스크린샷, 로그 기록 등)를 수집합니다.
   - **4단계**: 게임사에 공식적으로 이의를 제기하고, 필요한 경우 한국소비자원 등 관련 기관에 중재를 요청합니다.

5. **한